In [1]:
import gc
import pandas as pd

In [2]:
paths = './data_format1'
data = pd.read_csv(f'{paths}/user_log_format1.csv', dtype={'time_stamp':'str'})
data1 = pd.read_csv(f'{paths}/user_info_format1.csv')
data2 = pd.read_csv(f'{paths}/train_format1.csv')
submission = pd.read_csv(f'{paths}/test_format1.csv')
data_train = pd.read_csv('./data_format1/train_format2.csv')

In [3]:
data2['origin'] = 'train'
submission['origin'] = 'test'
matrix = pd.concat([data2, submission], ignore_index=True, sort=False)
matrix.drop(['prob'], axis=1, inplace=True)
matrix = matrix.merge(data1, on='user_id', how='left')
data.rename(columns={'seller_id':'merchant_id'}, inplace=True)

In [4]:
data.isnull().sum()

user_id            0
item_id            0
cat_id             0
merchant_id        1
brand_id       14700
time_stamp         1
action_type        1
dtype: int64

In [5]:
data['merchant_id'].fillna(1598.0,inplace=True)
data['time_stamp'].fillna('0929',inplace=True)
data['action_type'].fillna(0.0,inplace=True)

In [6]:
data.isnull().sum()

user_id            0
item_id            0
cat_id             0
merchant_id        0
brand_id       14700
time_stamp         0
action_type        0
dtype: int64

In [7]:
data['user_id'] = data['user_id'].astype('int32')
data['merchant_id'] = data['merchant_id'].astype('int32')
data['item_id'] = data['item_id'].astype('int32')
data['cat_id'] = data['cat_id'].astype('int32')
data['brand_id'].fillna(0, inplace=True)
data['brand_id'] = data['brand_id'].astype('int32')
data['time_stamp'] = pd.to_datetime(data['time_stamp'], format='%H%M')

matrix['age_range'].fillna(0, inplace=True)
matrix['gender'].fillna(2, inplace=True)
matrix['age_range'] = matrix['age_range'].astype('int8')
matrix['gender'] = matrix['gender'].astype('int8')
matrix['label'] = matrix['label'].astype('str')
matrix['user_id'] = matrix['user_id'].astype('int32')
matrix['merchant_id'] = matrix['merchant_id'].astype('int32')

del data1, data2
gc.collect()

0

In [8]:
#特征处理
groups = data.groupby(['user_id'])
temp = groups.size().reset_index().rename(columns={0:'u1'})
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.seconds/3600
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'u7', 1:'u8', 2:'u9', 3:'u10'})
matrix = matrix.merge(temp, on='user_id', how='left')


In [9]:
groups = data.groupby(['merchant_id'])
temp = groups.size().reset_index().rename(columns={0:'m1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={
    'user_id':'m2',
    'item_id':'m3', 
    'cat_id':'m4', 
    'brand_id':'m5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'m6', 1:'m7', 2:'m8', 3:'m9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')

In [10]:
temp = data_train[data_train['label']==-1].groupby(['merchant_id']).size().reset_index().rename(columns={0:'m10'})
matrix = matrix.merge(temp, on='merchant_id', how='left')

In [11]:
groups = data.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0:'um1'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={
    'item_id':'um2',
    'cat_id':'um3',
    'brand_id':'um4'
})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={
    0:'um5',
    1:'um6',
    2:'um7',
    3:'um8'
})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['time_stamp'].agg([('frist', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['frist']).dt.seconds/3600
temp.drop(['frist', 'last'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')

In [12]:
matrix['r1'] = matrix['u9']/matrix['u7'] #用户购买点击比
matrix['r2'] = matrix['m8']/matrix['m6'] #商家购买点击比
matrix['r3'] = matrix['um7']/matrix['um5'] #不同用户不同商家购买点击比

In [13]:
#train、test-setdata
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']
train_X.fillna(0, inplace=True)
test_data.fillna(0, inplace=True)

In [14]:
del temp, matrix
gc.collect()

90

In [15]:
#导入分析库
from sklearn.model_selection import train_test_split

In [28]:
import lightgbm as lgb

In [39]:
X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=.25)

In [52]:
# 创建模型，训练模型
gbm = lgb.LGBMClassifier(num_leaves=51,max_depth=10,boosting_type='gbdt',objective='binary',learning_rate=0.015,n_estimators=2000,
                         subsample=0.75,subsample_freq=2,reg_lambda=0.28,reg_alpha=0.12,colsample_bytree=0.8,min_child_samples=300,min_split_gain=0.1)

In [53]:
gbm.fit(X_train, y_train,eval_set=[(X_train, y_train), (X_valid, y_valid)],eval_metric='auc', early_stopping_rounds=100)

[1]	training's auc: 0.615783	training's binary_logloss: 0.229525	valid_1's auc: 0.611043	valid_1's binary_logloss: 0.230998
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.630443	training's binary_logloss: 0.229293	valid_1's auc: 0.622061	valid_1's binary_logloss: 0.23079
[3]	training's auc: 0.634307	training's binary_logloss: 0.229074	valid_1's auc: 0.623179	valid_1's binary_logloss: 0.230594
[4]	training's auc: 0.63698	training's binary_logloss: 0.228872	valid_1's auc: 0.623708	valid_1's binary_logloss: 0.230413
[5]	training's auc: 0.6407	training's binary_logloss: 0.228656	valid_1's auc: 0.627087	valid_1's binary_logloss: 0.230223
[6]	training's auc: 0.64299	training's binary_logloss: 0.228463	valid_1's auc: 0.631151	valid_1's binary_logloss: 0.230048
[7]	training's auc: 0.646131	training's binary_logloss: 0.228271	valid_1's auc: 0.633675	valid_1's binary_logloss: 0.229877
[8]	training's auc: 0.647281	training's binary_logloss: 0.228088	valid_1's 

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
               importance_type='split', learning_rate=0.015, max_depth=10,
               min_child_samples=300, min_child_weight=0.001,
               min_split_gain=0.1, n_estimators=2000, n_jobs=-1, num_leaves=51,
               objective='binary', random_state=None, reg_alpha=0.12,
               reg_lambda=0.28, silent=True, subsample=0.75,
               subsample_for_bin=200000, subsample_freq=2)

In [54]:
# 预测数据集
prob = gbm.predict_proba(test_data)

In [55]:
submission['prob'] = pd.Series(prob[:,1])
#submission.drop(['origin'], axis=1, inplace=True)
submission.to_csv('submission1.csv', index=False)